In [46]:
import os
import requests
from bs4 import BeautifulSoup
import textwrap
import pandas as pd
import lxml  
import time
from IPython.display import clear_output
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import re
from tqdm import tqdm

In [47]:
class keywords(object):
    smartphone = 'smartphone'
    notebook = 'notebook'

In [98]:
class Driver(object):
    def __init__(self, chromedriver_path, headless = True):
        options = Options()
        options.headless = headless
#         options.add_argument('--headless')
#         options.add_argument('window-size=1920x1080')
        
        self.driver = webdriver.Chrome(chromedriver_path, options = options)

    def getLink(self, link_text, waiting_time = 1, niters = 5, raise_error = True):
        for _ in range(niters):
            try:
                return self.driver.find_element_by_partial_link_text(link_text)
            except:
                sleep(waiting_time)
        
        if raise_error:
            raise(Exception("Driver: Failed to get link"))
        
    def getHTML(self, url, waiting_time = 2):
        self.driver.get(url)
        
        sleep(waiting_time)

        content = self.driver.page_source

        return BeautifulSoup(content, 'html.parser')
    
    def processRequest(self, url, soup_function, waiting_time = 1, niters = 1, raise_error = True, load_url = True):
        if load_url:
            self.driver.get(url)
            sleep(waiting_time)
        
        for _ in range(niters):
            try:
                soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                return soup_function(soup)
            except:
                sleep(waiting_time)
        
        if raise_error:
            raise(Exception("Driver: Failed to process request"))
        
chromedriver_path = r'C:\Users\Yuriy\Downloads\chromedriver_win32_4\chromedriver.exe'
driver = Driver(chromedriver_path, headless = False)

In [92]:
class DNS_shop_parser(object):
    def __init__(self, driver):
        self.driver = driver
        self.root_url = "https://www.dns-shop.ru"
        
        self.product_urls = {
                keywords.smartphone: "/".join((self.root_url, 'catalog/17a8a01d16404e77/smartfony')),
                keywords.notebook: "/".join((self.root_url, 'catalog/17a892f816404e77/noutbuki'))
            }
        
        self.link_keywords = {
                keywords.smartphone: ["Смартфон "],
                keywords.notebook: ["Ноутбук ", "Ультрабук ", "Нетбук "]
            }
        
        self.link_patterns = {
                keywords.smartphone: " .*?фон ",
                keywords.notebook: " .*?бук "
            }
        
        self.setDefaultLocation()
        
    def setDefaultLocation(self):
        default_city = "Москва" # Moscow
        
        soup = self.parseURL(self.root_url)
        
        if soup.find("div", {"class": "w-choose-city-widget-label"}).getText() == default_city:
            print('default location already set')
            return
        
        try:
            driver.getLink(link_text = "Выбрать другой").click(); sleep(2)
            driver.getLink(link_text = default_city,).click(); sleep(2)
            return
        except:
            raise(Exception("DNS_shop_parser: Failed to set default location"))
        
    def parseURL(self, url, waiting_time = 2):
        return self.driver.getHTML(url, waiting_time)
    
    def getPageLinks(self, product_keyword, page_num = 1):
        url = self.product_urls[product_keyword]
        pattern = self.link_patterns[product_keyword]
        
        soup = self.parseURL("/".join((url, f'?p={page_num}')))
        
        return soup.find_all("a", string = re.compile(pattern))
    
    def parseProductName(self, name, product_keyword):
        keywords = self.link_keywords[product_keyword]
        
        for keyword in keywords:
            if re.search(keyword, name) is not None:
                return name[name.find(keyword) + len(keyword):]
        
    def parseRawLinks(self, raw_links, product_keyword):
        hrefs = [self.root_url + link.get('href') for link in raw_links]
        names = [link.string for link in raw_links]
        names = list(map(lambda name: self.parseProductName(name, product_keyword), names))
        
        return [{'name': name, 'href': href} for (name, href) in zip(names, hrefs)]
    
    def getAllLinks(self, product_keyword, npages = 100):
        page_num = 1
        
        all_links = []
        
        for page_num in tqdm(range(1, npages + 1)):          
#             clear_output(wait = True)
#             print(f'current page = {page_num}')
            
            links = self.getPageLinks(product_keyword, page_num)
            
            if len(links) == 0:
                break
            
            all_links.extend(links)
            sleep(5)
            
            if page_num % 5 == 0:
                pd.DataFrame(self.parseRawLinks(all_links, product_keyword)).\
                        to_csv(f'{product_keyword}_pages.csv', encoding = 'utf-16')
            
#         clear_output(wait = True)
#         print(f'done! processed {page_num} pages')

        return self.parseRawLinks(all_links, product_keyword)   

parser = DNS_shop_parser(driver)

default location already set


In [93]:
smartphones = parser.getAllLinks(keywords.smartphone, npages = 100)
smartphones[:5], len(smartphones)

 36%|█████████████████████████████▏                                                   | 36/100 [06:49<12:07, 11.37s/it]


([{'name': 'DEXP AL240 8 ГБ красный',
   'href': 'https://www.dns-shop.ru/product/e9a278378a4e3330/4-smartfon-dexp-al240-8-gb-krasnyj/'},
  {'name': 'DEXP AL240 8 ГБ черный',
   'href': 'https://www.dns-shop.ru/product/9e857c598a4e3330/4-smartfon-dexp-al240-8-gb-cernyj/'},
  {'name': 'DEXP G253 8 ГБ золотистый',
   'href': 'https://www.dns-shop.ru/product/725b4a8375e23330/5-smartfon-dexp-g253-8-gb-zolotistyj/'},
  {'name': 'DEXP B355 8 ГБ золотистый',
   'href': 'https://www.dns-shop.ru/product/4f42f6d29a0b3330/545-smartfon-dexp-b355-8-gb-zolotistyj/'},
  {'name': 'DEXP B355 8 ГБ черный',
   'href': 'https://www.dns-shop.ru/product/0a0ff2499a0b3330/545-smartfon-dexp-b355-8-gb-cernyj/'}],
 635)

In [95]:
notebooks = parser.getAllLinks(keywords.notebook, npages = 100)
notebooks[:5], len(notebooks)

 41%|█████████████████████████████████▏                                               | 41/100 [08:05<11:38, 11.83s/it]


([{'name': 'Digma EVE 10 C301 черный',
   'href': 'https://www.dns-shop.ru/product/a716b761336f1b80/101-netbuk-digma-eve-10-c301-cernyj/'},
  {'name': 'ASUS Laptop E210MA-GJ001T синий',
   'href': 'https://www.dns-shop.ru/product/5c01fb1e84773332/116-noutbuk-asus-laptop-e210ma-gj001t-sinij/'},
  {'name': 'ASUS Laptop E210MA-GJ002T розовый',
   'href': 'https://www.dns-shop.ru/product/62131bcd84773332/116-noutbuk-asus-laptop-e210ma-gj002t-rozovyj/'},
  {'name': 'ASUS Laptop E210MA-GJ003T белый',
   'href': 'https://www.dns-shop.ru/product/62131bce84773332/116-noutbuk-asus-laptop-e210ma-gj003t-belyj/'},
  {'name': 'ASUS Laptop K540BA-GQ613 черный',
   'href': 'https://www.dns-shop.ru/product/af32176fcd753332/156-noutbuk-asus-laptop-k540ba-gq613-cernyj/'}],
 729)

In [101]:
class DNS_product_parser(object):    
    def __init__(self, driver):
        self.driver = driver
        self.root_url = "https://www.dns-shop.ru"

        self.topics_parsers = {
            'description': self.getDescription, 
            'characteristics': self.getCharacteristics, 
            'opinion': self.getOpinions, 
            'comment': self.getComments, 
            'price': self.getPrice,
            'rating': self.getRating,
        }
        
        self.setDefaultLocation()
        
    def setDefaultLocation(self):
        default_city = "Москва" # Moscow
        
        soup = self.parseURL(self.root_url)
        
        if soup.find("div", {"class": "w-choose-city-widget-label"}).getText() == default_city:
            print('default location already set')
            return
        
        try:
            driver.getLink(link_text = "Выбрать другой").click(); sleep(2)
            driver.getLink(link_text = default_city,).click(); sleep(2)
            return
        except:
            raise(Exception("DNS_product_parser: Failed to set default location"))
            
    def parseURL(self, url, waiting_time = 2):
        return self.driver.getHTML(url, waiting_time)
    
    def getPrice(self, link):
        url = link['href']

        soup_function = lambda soup: float(soup.find("span", {"class": re.compile(r'^product-card-price__current')})\
                                              .getText()[:-2].replace(' ', ''))
        price = self.driver.processRequest(url, soup_function, niters = 100, load_url = False)

        return {'price': price}
    
    def getRating(self, link):
        url = link['href']

        soup_function = lambda soup: float(soup.find("span", {"itemprop": "ratingValue"}).getText())
        average_rating = self.driver.processRequest(url, soup_function, niters = 3, raise_error = False, load_url = False)
        
        soup_function = lambda soup: int(soup.find("span", {"itemprop": "ratingCount"}).getText())
        reviews_num = self.driver.processRequest(url, soup_function, niters = 3, raise_error = False, load_url = False)
        
        return {'average_rating': average_rating, 'reviews_num': reviews_num}

    
    def getDescription(self, link):
        url = link['href'] + 'description'
        soup = self.parseURL(url)
        description_string = soup.find("div", {"class": "price-item-description"}).find('p').getText()

        return {'description': description_string}
    
    def getCharacteristics(self, link):
        url = link['href'] + 'characteristics'
        
        soup = self.parseURL(url)
        
        characteristics_table = soup.find('table')
        df = pd.read_html(str(characteristics_table))[0]
        df = df.drop(df[df[0] == df[1]].index)
        chars_dict = dict(zip(df[0],df[1]))

        return chars_dict

    def getOpinions(self, link):
        url = link['href'] + 'opinion'

        soup = self.parseURL(url)
        
        ratings_num = {}

        for i in range(1, 6):
            ratings_num["rating " + str(i)] = int(soup.find("div", {"class": "ow-counts"}).\
                                                  find("a", {"data-rating": str(i)}).getText().split()[0])
    
    def getComments(self, link):
        pass
    
    def parseTopics(self, links, topics = ['characteristics', 'rating', 'price']):
        links_values = pd.DataFrame()
        
        for link in tqdm(links):
            link_vals = {}
            
            brand_name, model_name = link['name'][: link['name'].find(' ')], link['name'][link['name'].find(' ') + 1:]

            for topic in topics:
                topic_parser = self.topics_parsers[topic]
                link_vals.update(topic_parser(link))
                
            clear_output(wait = True)
                
            links_values = links_values.append(link_vals, ignore_index = True)
            
            links_values.to_csv('notebooks.csv', encoding = 'utf-16')
            
        return links_values

product_parser = DNS_product_parser(driver)

default location already set


In [102]:
links_values = product_parser.parseTopics(notebooks)

100%|██████████████████████████████████████████████████████████████████████████████| 729/729 [1:59:54<00:00,  9.87s/it]


In [109]:
pd.read_csv('notebooks.csv', encoding = 'utf-16', index_col = 0)

,CrossFire/SLI-массив,SSHD накопитель (объем SSD буфера),average_rating,price,reviews_num,Автоматическое увеличение частоты,Архитектура процессора,Аудио интерфейсы,Беспроводные виды доступа в Интернет,Веб-камера,...,Цветовой охват,Цифровой блок клавиатуры,Частота,Частота оперативной памяти,Ширина,Год релиза,Количество слотов под модули памяти,"Особенности, дополнительно",Скорость сетевого адаптера,Технология динамического обновления экрана
0,нет,нет,1.0,17999.0,1.0,2.2 ГГц,GoldMont,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,"49% sRGB, 45% NTSC, 34% AdobeRGB",нет,1.1 ГГц,2133 МГц,268 мм,NaN,NaN,NaN,NaN,NaN
1,нет,нет,4.2,24999.0,17.0,2.8 ГГц,Goldmont Plus,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,45% NTSC,есть,1.1 ГГц,2400 МГц,279 мм,2020.0,интегрирована,NumberPad,нет,нет
2,нет,нет,4.7,24999.0,3.0,2.8 ГГц,Goldmont Plus,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,45% NTSC,есть,1.1 ГГц,2400 МГц,279 мм,2020.0,интегрирована,NumberPad,нет,нет
3,нет,нет,4.6,24999.0,9.0,2.8 ГГц,Goldmont Plus,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,45% NTSC,нет,1.1 ГГц,2133 МГц,279 мм,2020.0,интегрирована,NumberPad,нет,нет
4,нет,нет,3.8,28999.0,9.0,2.6 ГГц,Excavator,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,NaN,есть,2.3 ГГц,1866 МГц,381 мм,2020.0,1,NaN,нет,нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,нет,нет,NaN,313999.0,NaN,4.5 ГГц,Comet Lake,"3.5 мм jack (наушники), 3.5 мм jack (микрофонный)",Wi-Fi,есть,...,100% AdobeRGB,нет,2.6 ГГц,2666 МГц,358.5 мм,2020.0,2,NaN,1000 Мбит,нет
725,нет,нет,3.0,349999.0,2.0,4.5 ГГц,Comet Lake,"3.5 мм jack (наушники), 3.5 мм jack (микрофонный)",Wi-Fi,есть,...,100% AdobeRGB,нет,2.6 ГГц,2666 МГц,358.5 мм,2020.0,2,NaN,1000 Мбит,нет
726,нет,нет,NaN,379999.0,NaN,5 ГГц,Comet Lake,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,100% AdobeRGB,нет,2.6 ГГц,2933 МГц,358.5 мм,2020.0,2,NaN,1000 Мбит,нет
727,нет,нет,NaN,380999.0,NaN,4.5 ГГц,Comet Lake,"3.5 мм jack (наушники), 3.5 мм jack (микрофонный)",Wi-Fi,есть,...,100% AdobeRGB,нет,2.6 ГГц,2666 МГц,358.5 мм,2020.0,2,NaN,1000 Мбит,нет


In [119]:
product_parser.getRating(notebooks[0])

{'average_rating': 1.0, 'reviews_num': 1}

In [120]:
product_parser.getRating(smartphones[0])

{'average_rating': 3.7, 'reviews_num': 25}

In [123]:
product_parser.getPrice(notebooks[0])

{'price': 17999.0}

In [124]:
product_parser.getPrice(smartphones[0])

{'price': 1999.0}

In [125]:
product_parser.getDescription(notebooks[0])

{'description': 'Нетбук Digma на базе процессора Intel Celeron N3450 со встроенной графикой Intel HD Graphics 500. Компактное устройство, подходящее для просмотра видео, интернет-серфинга и выполнения повседневной работы.'}

In [126]:
product_parser.getDescription(smartphones[0])

{'description': 'Смартфон DEXP AL240 собран в корпусе, дизайн которого оформлен с помощью эффектного сочетания красного и черного цветов. Перед вами – универсальная модель от известного производителя. Благодаря поддержке стандарта 3G вы сможете использовать мобильное интернет-подключение. Дома и в общественных местах будет полезно подключение к сетям Wi-Fi. Модуль Bluetooth обеспечит вас возможностью коммутации с большим количеством портативных устройств. Смартфон DEXP AL240 оснащен 4-дюймовым экраном с разрешением 800x480. Можно использовать две SIM-карты. Важным элементом конструкции устройства для многих пользователей станет модуль GPS, обеспечивающий возможность использования навигационных приложений и функции геопозиционирования. Смартфон может работать автономно в течение времени, достигающего 560 ч. Максимальное время автономной работы в режиме разговора – 12 ч. Емкость аккумулятора смартфона равна 2800 мА·ч. Ширина, высота и толщина корпуса портативного устройства равны 63, 126

In [127]:
product_parser.getCharacteristics(notebooks[0])

{'Гарантия': '12 мес.',
 'Страна-производитель': 'Китай',
 'Тип устройства': 'нетбук',
 'Операционная система': 'Windows 10 Home',
 'Модель': 'Digma EVE 10 C301',
 'Код производителя': 'ES1050EW',
 'Игровой ноутбук': 'нет',
 'Цвет верхней крышки': 'черный',
 'Материал корпуса': 'пластик',
 'Конструктивное исполнение': 'классический',
 'Цифровой блок клавиатуры': 'нет',
 'Подсветка клавиш': 'нет',
 'Тип экрана': 'IPS',
 'Диагональ экрана (дюйм)': '10.1"',
 'Разрешение экрана': '1280x800',
 'Название формата': 'HD',
 'Время отклика пикселя, мс': '30',
 'Плотность пикселей': '149 PPI',
 'Максимальная частота обновления экрана': '60 Гц',
 'Цветовой охват': '49% sRGB, 45% NTSC, 34% AdobeRGB',
 'Покрытие экрана': 'матовое',
 'Сенсорный экран': 'нет',
 'Производитель процессора': 'Intel',
 'Линейка процессора': 'Celeron',
 'Модель процессора': 'N3450',
 'Количество ядер процессора': '4',
 'Максимальное число потоков': '4',
 'Частота': '1.1 ГГц',
 'Автоматическое увеличение частоты': '2.2 ГГц'

In [128]:
product_parser.getCharacteristics(smartphones[0])

{'Гарантия': '12 мес.',
 'Страна-производитель': 'Китай',
 'Год выпуска': '2019',
 'Цвет задней панели': 'красный',
 'Цвет передней панели': 'черный',
 'Цвет граней': 'красный',
 'Цвет, заявленный производителем': 'красный',
 'Поддержка сетей 2G': 'GSM 900, GSM 1800, GSM 1900, GSM 850',
 'Поддержка сетей 3G': 'WCDMA 900, WCDMA 2100',
 'Поддержка сетей 4G (LTE)': 'нет',
 'Количество SIM-карт': '2 SIM',
 'Диагональ экрана (дюйм)': '4"',
 'Разрешение экрана': '800x480',
 'Плотность пикселей': '233 ppi',
 'Технология изготовления экрана': 'TN',
 'Частота обновления экрана': '60 Гц',
 'Материал корпуса': 'пластик',
 'Степень защиты IP': 'нет',
 'Версия ОС': 'Android 8.1 Oreo Go',
 'Производитель процессора': 'MediaTek',
 'Модель процессора': 'MediaTek MT6580M',
 'Количество ядер': '4',
 'Частота работы процессора': '1 ГГц',
 'Конфигурация процессора': '4x Cortex-A7 1 ГГц',
 'Графический ускоритель': 'Mali-400 MP2',
 'Объем оперативной памяти': '512 МБ',
 'Объем встроенной памяти': '8 ГБ',
 